In [1]:
import pandas as pd

# Анализ данных

In [2]:
fields = ['Product', 'Summary']
data = pd.read_csv("bugs-2018-04-05.csv", usecols=fields)
data.head()

,Product,Summary
0,Thunderbird,e-mail tag is not visible when thread is colla...
1,Calendar,Make use of Assert.jsm in xpcshell tests
2,Core,Add touch-action regions to the layer EventReg...
3,Core,Transition more fields of FrameMetrics to use ...
4,Calendar,Getter fails in calender-migration-dialog on f...


In [3]:
data.describe()

,Product,Summary
count,10000,10000
unique,78,9952
top,Core,reboots-scl1
freq,2122,8


In [4]:
import re
RE_WORDS = re.compile(r'''
    # Find words in a string. Order matters!
    [A-Z]+(?=[A-Z][a-z]) |  # All upper case before a capitalized word
    [A-Z]?[a-z]+ |  # Capitalized words / all lower case
    [A-Z]+ |  # All upper case
    \d+  # Numbers
''', re.VERBOSE)

def clean_text(text):
    text = text.replace(r"[^A-Za-z0-9]", " ")
    text = re.sub("[\]\[_'\\\/.,;:\-()<>\"]", " ", text)
    text = ' '.join(word for word in RE_WORDS.findall(text) if len(word)>2 or word.isupper())
    text = text.lower()
    return text

data['CleanSummary'] = data.apply(lambda x: clean_text(x['Summary']), axis=1)

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

data['Tokens'] = data['CleanSummary'].apply(tokenizer.tokenize)
data.to_pickle('dataframe.pkl')

data.tail()


,Product,Summary,CleanSummary,Tokens
9995,Webmaker,[Meta] Solution to user's preference locale se...,meta solution user preference locale setting a...,"[meta, solution, user, preference, locale, set..."
9996,Webmaker,Figure out a way to instantiate variables in a...,figure out way instantiate variables angular,"[figure, out, way, instantiate, variables, ang..."
9997,Webmaker,Change SECRET_SESSION to match other apps,change secret session match other apps,"[change, secret, session, match, other, apps]"
9998,Webmaker,Missing selectize in define details-form,missing selectize define details form,"[missing, selectize, define, details, form]"
9999,Webmaker,Popcorn video from three months ago - Text fie...,popcorn video from three months ago text field...,"[popcorn, video, from, three, months, ago, tex..."


In [5]:
all_words = [word for tokens in data['Tokens'] for word in tokens]
sentence_lengths = [len(tokens) for tokens in data['Tokens']]
VOCAB = sorted(list(set(all_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_words), len(VOCAB)))
print("Max sentence length is %s" % max(sentence_lengths))

83058 words total, with a vocabulary size of 8622
Max sentence length is 37


In [6]:
categories = {}
keys = enumerate(data['Product'].unique())
for key, value in keys:
    categories[value] = key + 1
data['ProductIndex'] = data['Product'].map(categories)
print(categories)

# Этот код - пробовал объединять схожие категории, не нужен сейчас
#keys = sorted(list(set(data['Product'])))
#print(keys)
#data['ProductIndex'] = [keys.index(key) for key in data['Product']]

#joinedKeys = {}
#index = 6
#for key in keys:
#    k = key.lower()
#    if "mozilla" in k or "bugzilla" in k or "web" in k:
#        joinedKeys.setdefault(key, 0)
#    elif "firefox" in k:
#        joinedKeys.setdefault(key, 1)
#    elif "penelope" in k or "documentation" in k:
#        joinedKeys.setdefault(key, 2)
#    elif "core" in k or "task" in k or "socorro" in k or "seamonkey" in k or "tool" in k or "develop" in k:
#        joinedKeys.setdefault(key, 3)
#    elif "testing" in k:
#        joinedKeys.setdefault(key, 4)
#    elif "infractructure" in k or "release" in k or "tracking" in k:
#        joinedKeys.setdefault(key, 5)
    # next is for other - too litle data for that
#    elif "community building" in k or "directory" in k or "skywriter" in k or "shield" in k or "nspr" in k \
#            or "nss" in k or "privacy" in k or "snippets" in k or "jss" in k:
#        joinedKeys.setdefault(key, 100)
#    else:
#        joinedKeys.setdefault(key, index)
#        index += 1

#joinedKeysList = list(joinedKeys.keys())
#print(len(joinedKeysList))
#print(len(set(joinedKeys.values())))
#print(joinedKeys)
#data['JoinedProductIndex'] = [joinedKeys[key] for key in data['Product']]
data.to_pickle('dataframe.pkl')
data.tail()

{'Thunderbird': 1, 'Calendar': 2, 'Core': 3, 'Firefox OS': 4, 'www.mozilla.org': 5, 'Toolkit': 6, 'Firefox': 7, 'SeaMonkey': 8, 'Chat Core': 9, 'Firefox for Android Graveyard': 10, 'Mozilla Labs': 11, 'Firefox Build System': 12, 'Tree Management Graveyard': 13, 'Toolkit Graveyard': 14, 'Participation Infrastructure': 15, 'Developer Documentation': 16, 'MailNews Core': 17, 'Mozilla Localizations': 18, 'Webtools': 19, 'Instantbird': 20, 'Cloud Services': 21, 'Firefox for Metro': 22, 'Firefox for Android': 23, 'Firefox Health Report Graveyard': 24, 'Other Applications': 25, 'addons.mozilla.org Graveyard': 26, 'Websites Graveyard': 27, 'support.mozilla.org': 28, 'Firefox Affiliates Graveyard': 29, 'Bugzilla': 30, 'Websites': 31, 'Infrastructure & Operations': 32, 'mozilla.org Graveyard': 33, 'Release Engineering': 34, 'Tracking': 35, 'Firefox for iOS': 36, 'Hello (Loop)': 37, 'Mozilla QA': 38, 'Mozilla QA Graveyard': 39, 'Core Graveyard': 40, 'Tech Evangelism': 41, 'Testing': 42, 'develope

,Product,Summary,CleanSummary,Tokens,ProductIndex
9995,Webmaker,[Meta] Solution to user's preference locale se...,meta solution user preference locale setting a...,"[meta, solution, user, preference, locale, set...",51
9996,Webmaker,Figure out a way to instantiate variables in a...,figure out way instantiate variables angular,"[figure, out, way, instantiate, variables, ang...",51
9997,Webmaker,Change SECRET_SESSION to match other apps,change secret session match other apps,"[change, secret, session, match, other, apps]",51
9998,Webmaker,Missing selectize in define details-form,missing selectize define details form,"[missing, selectize, define, details, form]",51
9999,Webmaker,Popcorn video from three months ago - Text fie...,popcorn video from three months ago text field...,"[popcorn, video, from, three, months, ago, tex...",51


In [7]:
# Код для выкидывания данных по редким категориям - тоже не нужен

#keys = sorted(list(set(data['Product'])))
#print("Number of products with more then 300 records")
#prodCount = 0
#keyCounts = {}
#for key in keys:
#    count = len(data[(data['Product'] == key)])
#    if count > 300:
#        keyCounts[key] = count
#        prodCount +=1
    
#keyCounts = sorted(keyCounts.items(), key=operator.itemgetter(1))
#print(keyCounts)
#print(keyCounts[0][0])
#print(prodCount)

In [8]:
#print(keyCounts.keys())
#for key in keys:
#    if key not in keyCounts.keys():
#        data = data[(data['Product'] != key)]
#        print(key)
        
#data.tail()
        

In [9]:
# Код создания индексов для оставшихся top 10 категорий - эже не нужен, так как ничего не выкидываю

#data = pd.read_pickle('dataframe.pkl')
#topKeys = list(keyCounts.keys())
#print(topKeys)
#data['Top10ProductIndex'] = [topKeys.index(key) for key in data['Product']]
#data.to_pickle('dataframe.pkl')
#data.head()


Создаю сеть

In [1]:
import pandas as pd
import random

from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Input, Dense, Embedding, Conv1D, Conv2D, MaxPool2D, MaxPool1D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model
from keras.models import Sequential
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

from sklearn.utils import shuffle

data = pd.read_pickle('dataframe.pkl')
data = data.sample(frac=1).reset_index(drop=True)

all_words = [word for tokens in data['Tokens'] for word in tokens]
sentence_lengths = [len(tokens) for tokens in data['Tokens']]
max_sentence_length = max(sentence_lengths)
vocabulary = sorted(list(set(all_words)))
words_total=len(all_words)
vocabulary_size=len(vocabulary)
print("%s всего слов, размер словаря %s" % (words_total, vocabulary_size))
print("Макс длина текстов %s" % max_sentence_length)

summaries = data['Tokens'].tolist()
products = data['ProductIndex'].tolist()

products_number=len(list(set(products)))+1 # так как индекса идут с 1, для top10 было с 0 и прибавлять ничего не надо
print("Всего категорий:", products_number)
tokenizer = Tokenizer(vocabulary_size)
tokenizer.fit_on_texts(summaries)
textSequences = tokenizer.texts_to_sequences(summaries)

textSequences = pad_sequences(textSequences, maxlen=max_sentence_length)
#[random.shuffle(sentence) for sentence in textSequences]

X_train, X_test, y_train, y_test = train_test_split(textSequences, products, random_state=0)
print(len(X_train))
print(len(X_test))
print(X_train[0:5])
print(X_test[0:5])

print('Размерность X_train:', X_train.shape)
print('Размерность X_test:', X_test.shape)

y_train = to_categorical(y_train, products_number)
y_test = to_categorical(y_test, products_number)
print('y_train форма:', y_train.shape)
print('y_test форма:', y_test.shape)

embedding_dim = 256
filter_sizes = [3,4,5]
num_filters = 512
drop = 0.5

epochs = 10
batch_size = 150

inputs = Input(shape=(max_sentence_length,), dtype='int32')
embedding = Embedding(input_dim=vocabulary_size+1, output_dim=embedding_dim, input_length=max_sentence_length)(inputs)
reshape = Reshape((max_sentence_length,embedding_dim,1))(embedding)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal',
                activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal',
                activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal',
                activation='relu')(reshape)

maxpool_0 = MaxPool2D(pool_size=(max_sentence_length - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(max_sentence_length - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(max_sentence_length - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(drop)(flatten)
output = Dense(products_number, activation='softmax')(dropout)

model = Model(inputs=inputs, outputs=output)

checkpoint = ModelCheckpoint('weights.{epoch:03d}-{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True,
                             mode='auto')
#adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

# А здесь особенность keras - можно использовать и binary  и categorical - но тогда надо правильно задавать metrics
# Вот тут про это подробно
# https://stackoverflow.com/questions/42081257/keras-binary-crossentropy-vs-categorical-crossentropy-performance
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
print("Traning Model...")
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))  # starts training



/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using TensorFlow backend.


83058 всего слов, размер словаря 8622
Макс длина текстов 37
Всего категорий: 79


7500
2500
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0  321 1468 3177   88
  1723  278  327 1024    5  279    8 1840   65]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    7 3581 3582    2  573 1393]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0  469 6052 2213  759  999  102 1944 6053]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0  225  272   98   57    1  270    3  789]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0   77
  1330  134   10    7   31    1  553  500  602]]
[[   0    0    0    0    0   

Traning Model...


...

6900/7500 [==========================>...] - ETA: 5s - loss: 0.1222 - acc: 0.9833

7050/7500 [===========================>..] - ETA: 4s - loss: 0.1207 - acc: 0.9837

7200/7500 [===========================>..] - ETA: 2s - loss: 0.1204 - acc: 0.9839

7350/7500 [============================>.] - ETA: 1s - loss: 0.1208 - acc: 0.9839

7500/7500 [==============================] - 72s 10ms/step - loss: 0.1203 - acc: 0.9839 - val_loss: 1.7301 - val_acc: 0.6296


In [2]:
import pandas as pd
import random

from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Input, Dense, Embedding, Conv1D, Conv2D, MaxPool2D, MaxPool1D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model
from keras.models import Sequential
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

from sklearn.utils import shuffle

data = pd.read_pickle('dataframe.pkl')
data = data.sample(frac=1).reset_index(drop=True)

all_words = [word for tokens in data['Tokens'] for word in tokens]
sentence_lengths = [len(tokens) for tokens in data['Tokens']]
max_sentence_length = max(sentence_lengths)
vocabulary = sorted(list(set(all_words)))
words_total=len(all_words)
vocabulary_size=len(vocabulary)
print("%s всего слов, размер словаря %s" % (words_total, vocabulary_size))
print("Макс длина текстов %s" % max_sentence_length)

summaries = data['Tokens'].tolist()
products = data['ProductIndex'].tolist()

products_number=len(list(set(products)))+1 # так как индекса идут с 1, для top10 было с 0 и прибавлять ничего не надо
print("Всего категорий:", products_number)
tokenizer = Tokenizer(vocabulary_size)
tokenizer.fit_on_texts(summaries)
textSequences = tokenizer.texts_to_sequences(summaries)

textSequences = pad_sequences(textSequences, maxlen=max_sentence_length)
#[random.shuffle(sentence) for sentence in textSequences]

X_train, X_test, y_train, y_test = train_test_split(textSequences, products, random_state=0)
print(len(X_train))
print(len(X_test))
print(X_train[0:5])
print(X_test[0:5])

print('Размерность X_train:', X_train.shape)
print('Размерность X_test:', X_test.shape)

y_train = to_categorical(y_train, products_number)
y_test = to_categorical(y_test, products_number)
print('y_train форма:', y_train.shape)
print('y_test форма:', y_test.shape)

embedding_dim = 256
filter_sizes = [3,4,5]
num_filters = 512
drop = 0.5

epochs = 10
batch_size = 150

inputs = Input(shape=(max_sentence_length,), dtype='int32')
embedding = Embedding(input_dim=vocabulary_size+1, output_dim=embedding_dim, input_length=max_sentence_length)(inputs)

conv_0 = Conv1D(num_filters, kernel_size=filter_sizes[0], padding='valid', kernel_initializer='normal',
                activation='relu')(embedding)
conv_1 = Conv1D(num_filters, kernel_size=filter_sizes[1], padding='valid', kernel_initializer='normal',
                activation='relu')(embedding)
conv_2 = Conv1D(num_filters, kernel_size=filter_sizes[2], padding='valid', kernel_initializer='normal',
                activation='relu')(embedding)
#conv_3 = Conv1D(num_filters, 8, activation='relu')(embedding)
#conv_4 = Conv1D(num_filters, 5, activation='relu')(embedding)
#conv_5 = Conv1D(num_filters, 3, activation='relu')(embedding)

maxpool_0 = MaxPool1D(pool_size=(max_sentence_length - filter_sizes[0] + 1), strides=1, padding='valid')(conv_0)
maxpool_1 = MaxPool1D(pool_size=(max_sentence_length - filter_sizes[1] + 1), strides=1, padding='valid')(conv_1)
maxpool_2 = MaxPool1D(pool_size=(max_sentence_length - filter_sizes[2] + 1), strides=1, padding='valid')(conv_2)
#maxpool_3 = MaxPool1D(pool_size=8)(conv_3)
#maxpool_4 = MaxPool1D(pool_size=5)(conv_4)
#maxpool_5 = MaxPool1D(pool_size=3)(conv_5)

concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(drop)(flatten)
output = Dense(products_number, activation='softmax')(dropout)

model = Model(inputs=inputs, outputs=output)


#model = Sequential()
#model.add(Embedding(vocabulary_size+1, embedding_dim, input_length=max_sentence_length))
#model.add(Conv1D(num_filters, 7, activation='relu'))
#model.add(MaxPooling1D(3))
#model.add(Conv1D(num_filters, 3, activation='relu'))
#model.add(MaxPooling1D(3))
#model.add(Conv1D(num_filters, 3, activation='relu'))
#model.add(Conv1D(num_filters, 3, activation='relu'))
#model.add(GlobalAveragePooling1D())
#model.add(Dropout(drop))
#model.add(Dense(products_number, activation='softmax'))



checkpoint = ModelCheckpoint('weights.{epoch:03d}-{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True,
                             mode='auto')
#adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

# А здесь особенность keras - можно использовать и binary  и categorical - но тогда надо правильно задавать metrics
# Вот тут про это подробно
# https://stackoverflow.com/questions/42081257/keras-binary-crossentropy-vs-categorical-crossentropy-performance
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
print("Traning Model...")
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))  # starts training



83058 всего слов, размер словаря 8622
Макс длина текстов 37
Всего категорий: 79


7500
2500
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0   49   60  247  437 2016 5871    1  274]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
  4904 3572 1228    3 4905 1228  224 1156 1799]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    7 1938 1785  571   84  733]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0  541    8 2222  268  411   90]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0  183  209  363    4  159  936    4   51
   936   51  936    4  909  189 1192  403  730]]
[[   0    0    0    0    0   

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


Traning Model...


Train on 7500 samples, validate on 2500 samples
Epoch 1/10


...

6900/7500 [==========================>...] - ETA: 5s - loss: 0.1163 - acc: 0.9859

7050/7500 [===========================>..] - ETA: 3s - loss: 0.1191 - acc: 0.9854

7200/7500 [===========================>..] - ETA: 2s - loss: 0.1204 - acc: 0.9851

7350/7500 [============================>.] - ETA: 1s - loss: 0.1206 - acc: 0.9850

7500/7500 [==============================] - 70s 9ms/step - loss: 0.1197 - acc: 0.9852 - val_loss: 1.8513 - val_acc: 0.6288


In [ ]:
from keras_text.data import Dataset